### 0. 基础环境

In [1]:
# 环境变量
from dotenv import load_dotenv
load_dotenv(dotenv_path=".baidu_env")
# LLM
from langchain_community.llms.baidu_qianfan_endpoint import QianfanLLMEndpoint
# 千帆大模型
from langchain_community.chat_models.baidu_qianfan_endpoint import QianfanChatEndpoint
# 嵌入模型
from langchain_community.embeddings import QianfanEmbeddingsEndpoint
# LLM
llm = QianfanLLMEndpoint(model="ERNIE-4.0-8K")
# 模型构建
chat = QianfanChatEndpoint(model="ERNIE-4.0-8K")

embed = QianfanEmbeddingsEndpoint(model='bge-large-zh')

### 1.  基础检索器

In [2]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter

D:\software\anaconda3\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
# 实例化在线资源加载
web_loader = WebBaseLoader(web_path="https://baike.baidu.com/item/%E6%AF%9B%E6%B3%BD%E4%B8%9C")

In [4]:
# 加载资源
web_docs = web_loader.load()

In [8]:
# 初始化 spliter
spliter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)

In [9]:
# 切分文档
docs = spliter.split_documents(documents=web_docs)

In [10]:
db = Chroma.from_documents(documents=docs, embedding=embed, persist_directory="./chroma_data")

[INFO][2024-08-29 21:09:24.976] oauth.py:228 [t:16052]: trying to refresh access_token for ak `i4ba2u***`
[INFO][2024-08-29 21:09:36.382] oauth.py:243 [t:16052]: sucessfully refresh access_token


In [11]:
db

In [12]:
# 默认的度量标准是 欧式距离 (不开平方)
results = db.similarity_search_with_score(query="毛泽东是谁？", k=2)

In [13]:
results

[(Document(metadata={'description': '毛泽东（1893年12月26日-1976年9月9日），字润之（原作咏芝，后改润芝），笔名子任。湖南湘潭人。中国人民的领袖、伟大的马克思主义者，伟大的无产阶级革命家、战略家、中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人、理论家，是马克思主义中国化的伟大开拓者、中国社会主义现代化建设事业的伟大奠基者，是近代以来中国伟大的爱国者和民族英雄，是党的第一代中央领导集体的核心，是领导中国人民彻底改变自己命运和国家面貌的一代伟人，是为世界被压迫民族的解放和人类进步事业作出重大贡献的伟大国际主义者。1949至1976年，毛泽东担任中华人民共和国最高领导人。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。因毛泽东担任过的主要职务几乎全部称为主席，所以也被人们尊称为“毛主席”。毛泽东被视为现代世界历史中最重要的人物之一，《时代》杂志也将他评为20世纪最具影响1', 'language': 'No language found.', 'source': 'https://baike.baidu.com/item/%E6%AF%9B%E6%B3%BD%E4%B8%9C', 'title': '毛泽东_百度百科'}, page_content='毛泽东_百度百科 网页新闻贴吧知道网盘图片视频地图文库资讯采购百科百度首页登录注册进入词条全站搜索帮助首页秒懂百科特色百科知识专题加入百科百科团队权威合作个人中心收藏查看我的收藏0有用+10毛泽东[máo zé dōng]播报上传视频伟大的无产阶级革命家毛泽东（1893年12月26日-1976年9月9日），字润之（原作咏芝，后改润芝）， [32]笔名子任。湖南湘潭人。中国人民的领袖、伟大的马克思主义者，伟大的无产阶级革命家、战略家、中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人、理论家，是马克思主义中国化的伟大开拓者、中国社会主义现代化建设事业的伟大奠基者，是近代以来中国伟大的爱国者和民族英雄，是党的第一代中央领导集体的核心，是领导中国人民彻底改变自己命运和国家面貌的一代伟人，是为世界被压迫民族的解放和人类进步事业作出重大贡献的伟大国际主义者。 [36]1949至1976年，毛泽东担任中华人民共和国最高领导人。他

$ relevance_score = 1 - \frac{distance}{\sqrt{2}} $

In [14]:
db.similarity_search_with_relevance_scores(query="毛泽东是谁？", k=2, score_threshold=0.65)

[(Document(metadata={'description': '毛泽东（1893年12月26日-1976年9月9日），字润之（原作咏芝，后改润芝），笔名子任。湖南湘潭人。中国人民的领袖、伟大的马克思主义者，伟大的无产阶级革命家、战略家、中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人、理论家，是马克思主义中国化的伟大开拓者、中国社会主义现代化建设事业的伟大奠基者，是近代以来中国伟大的爱国者和民族英雄，是党的第一代中央领导集体的核心，是领导中国人民彻底改变自己命运和国家面貌的一代伟人，是为世界被压迫民族的解放和人类进步事业作出重大贡献的伟大国际主义者。1949至1976年，毛泽东担任中华人民共和国最高领导人。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。因毛泽东担任过的主要职务几乎全部称为主席，所以也被人们尊称为“毛主席”。毛泽东被视为现代世界历史中最重要的人物之一，《时代》杂志也将他评为20世纪最具影响1', 'language': 'No language found.', 'source': 'https://baike.baidu.com/item/%E6%AF%9B%E6%B3%BD%E4%B8%9C', 'title': '毛泽东_百度百科'}, page_content='毛泽东_百度百科 网页新闻贴吧知道网盘图片视频地图文库资讯采购百科百度首页登录注册进入词条全站搜索帮助首页秒懂百科特色百科知识专题加入百科百科团队权威合作个人中心收藏查看我的收藏0有用+10毛泽东[máo zé dōng]播报上传视频伟大的无产阶级革命家毛泽东（1893年12月26日-1976年9月9日），字润之（原作咏芝，后改润芝）， [32]笔名子任。湖南湘潭人。中国人民的领袖、伟大的马克思主义者，伟大的无产阶级革命家、战略家、中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人、理论家，是马克思主义中国化的伟大开拓者、中国社会主义现代化建设事业的伟大奠基者，是近代以来中国伟大的爱国者和民族英雄，是党的第一代中央领导集体的核心，是领导中国人民彻底改变自己命运和国家面貌的一代伟人，是为世界被压迫民族的解放和人类进步事业作出重大贡献的伟大国际主义者。 [36]1949至1976年，毛泽东担任中华人民共和国最高领导人。他

In [15]:
results[0][0].page_content

'毛泽东_百度百科 网页新闻贴吧知道网盘图片视频地图文库资讯采购百科百度首页登录注册进入词条全站搜索帮助首页秒懂百科特色百科知识专题加入百科百科团队权威合作个人中心收藏查看我的收藏0有用+10毛泽东[máo zé dōng]播报上传视频伟大的无产阶级革命家毛泽东（1893年12月26日-1976年9月9日），字润之（原作咏芝，后改润芝）， [32]笔名子任。湖南湘潭人。中国人民的领袖、伟大的马克思主义者，伟大的无产阶级革命家、战略家、中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人、理论家，是马克思主义中国化的伟大开拓者、中国社会主义现代化建设事业的伟大奠基者，是近代以来中国伟大的爱国者和民族英雄，是党的第一代中央领导集体的核心，是领导中国人民彻底改变自己命运和国家面貌的一代伟人，是为世界被压迫民族的解放和人类进步事业作出重大贡献的伟大国际主义者。 [36]1949至1976年，毛泽东担任中华人民共和国最高领导人。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。'

### 插曲：相似度如何计算？

In [16]:
import numpy as np

In [17]:
v1 = embed.embed_query(text="毛泽东_百度百科 网页新闻贴吧知道网盘图片视频地图文库资讯采购百科百度首页登录注册进入词条全站搜索帮助首页秒懂百科特色百科知识专题加入百科百科团队权威合作个人中心收藏查看我的收藏0有用+10毛泽东[máo zé dōng]播报上传视频伟大的无产阶级革命家毛泽东（1893年12月26日-1976年9月9日），字润之（原作咏芝，后改润芝）， [33]笔名子任。湖南湘潭人。中国人民的领袖，伟大的马克思主义者，伟大的无产阶级革命家 [23]、战略家、理论家，中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人，马克思主义中国化的伟大开拓者，近代以来中国伟大的爱国者和民族英雄，中国共产党第一代中央领导集体的核心，领导中国人民彻底改变自己命运和国家面貌的一代伟人 [16] [23]。1949至1976年，毛泽东担任中华人民共和国最高领导人。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。")

In [18]:
v2 = embed.embed_query(text="毛泽东是谁？")

In [19]:
import numpy as np

In [20]:
v1 = np.array(v1)
v2 = np.array(v2)

In [21]:
distance = ((v1 - v2) ** 2).sum()

In [22]:
distance

0.4651953909711565

In [23]:
1 - distance / 2 ** 0.5

0.6710571844675681

### 基本检索

In [45]:
# 注意传参数 带阈值参数
retriever = db.as_retriever(
    # 距离度量标准，similarity 默认：欧式距离
    search_type ="similarity_score_threshold",
    # 搜索参数
    search_kwargs={"k": 10, "score_threshold": 0.30}
)

In [46]:
results = retriever.invoke("毛主席是谁？")

In [47]:
len(results)

10

### 多重提问检索
- 用户的提问可能比较偏门或是怪异，导致了不能命中核心参考信息！！
- 根据用户的提问，我们通过大模型给他生成几个类似提问，然后再去检索！！
- 简单来说，把用户的提问，换几个说法，再去提问，提高命中率！！
- 潜在好处：
  - 增加检索的覆盖面
  - 提高准确率
  - 适用用户的多样性

In [48]:
from langchain.retrievers import MultiQueryRetriever

In [49]:
prompt = """
You are an AI language model assistant. Your task is \n    to generate 3 different versions of the given user \n    question to retrieve relevant documents from a vector  database. \n    By generating multiple perspectives on the user question, \n    your goal is to help the user overcome some of the limitations \n    of distance-based similarity search. Provide these alternative \n    questions separated by newlines. Original question: {question}
"""

In [50]:
print(prompt)


You are an AI language model assistant. Your task is 
    to generate 3 different versions of the given user 
    question to retrieve relevant documents from a vector  database. 
    By generating multiple perspectives on the user question, 
    your goal is to help the user overcome some of the limitations 
    of distance-based similarity search. Provide these alternative 
    questions separated by newlines. Original question: {question}



In [51]:
"""
Given a query, use an LLM to write a set of queries. 借助大模型，把用户的提问生成一些列的类似提问

Retrieve docs for each query. 针对每个类似提问进行查询

Return the unique union of all retrieved docs. 把查到的结果去重，然后返回
"""
# 初始化一个多重提问检索器
mq_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=llm)

In [52]:
import logging

In [53]:
# logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [54]:
results = mq_retriever.invoke(input="毛泽东为什么又被叫做毛主席？")

In [55]:
len(results)

11

In [56]:
results

[Document(metadata={'description': '毛泽东（1893年12月26日-1976年9月9日），字润之（原作咏芝，后改润芝），笔名子任。湖南湘潭人。中国人民的领袖、伟大的马克思主义者，伟大的无产阶级革命家、战略家、中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人、理论家，是马克思主义中国化的伟大开拓者、中国社会主义现代化建设事业的伟大奠基者，是近代以来中国伟大的爱国者和民族英雄，是党的第一代中央领导集体的核心，是领导中国人民彻底改变自己命运和国家面貌的一代伟人，是为世界被压迫民族的解放和人类进步事业作出重大贡献的伟大国际主义者。1949至1976年，毛泽东担任中华人民共和国最高领导人。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。因毛泽东担任过的主要职务几乎全部称为主席，所以也被人们尊称为“毛主席”。毛泽东被视为现代世界历史中最重要的人物之一，《时代》杂志也将他评为20世纪最具影响1', 'language': 'No language found.', 'source': 'https://baike.baidu.com/item/%E6%AF%9B%E6%B3%BD%E4%B8%9C', 'title': '毛泽东_百度百科'}, page_content='[36]1949至1976年，毛泽东担任中华人民共和国最高领导人。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。 [33-34]因毛泽东担任过的主要职务几乎全部称为主席，所以也被人们尊称为“毛主席”。毛泽东被视为现代世界历史中最重要的人物之一，《时代》杂志也将他评为20世纪最具影响100人之一。中文名毛泽东别\xa0\xa0\xa0\xa0名石三伢子、二十八画生 等国\xa0\xa0\xa0\xa0籍中国民\xa0\xa0\xa0\xa0族汉族出生地湖南省长沙府湘潭县韶山冲出生日期1893年12月26日逝世日期1976年09月09日毕业院校湖南省立第一师范学校代表作品矛盾论、实践论、论持久战、沁园春·雪主要成就领导夺取中国新民主主义革命的胜利在中国确立社会主义基本制度 [22]毛泽东思想的主要创立者信\xa0\xa0\xa0\xa0仰马克思主义目录1人物生平2大事年表▪1893-1910年▪191

In [ ]:
### 分析prompt

In [57]:
from langchain.retrievers.multi_query import DEFAULT_QUERY_PROMPT

In [58]:
DEFAULT_QUERY_PROMPT

PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is \n    to generate 3 different versions of the given user \n    question to retrieve relevant documents from a vector  database. \n    By generating multiple perspectives on the user question, \n    your goal is to help the user overcome some of the limitations \n    of distance-based similarity search. Provide these alternative \n    questions separated by newlines. Original question: {question}')

In [59]:
print(DEFAULT_QUERY_PROMPT.template)

You are an AI language model assistant. Your task is 
    to generate 3 different versions of the given user 
    question to retrieve relevant documents from a vector  database. 
    By generating multiple perspectives on the user question, 
    your goal is to help the user overcome some of the limitations 
    of distance-based similarity search. Provide these alternative 
    questions separated by newlines. Original question: {question}


### 自定义多重提问 PromptTempalte

In [60]:
from langchain_core.prompts import PromptTemplate

In [64]:
my_multi_query_prompt = PromptTemplate.from_template(template="""
    You are an AI language model assistant. Your task is 
    to generate 5 different versions of the given user 
    question to retrieve relevant documents from a vector  database. 
    By generating multiple perspectives on the user question, 
    your goal is to help the user overcome some of the limitations 
    of distance-based similarity search. Provide these alternative 
    questions separated by newlines. Original question: {question}""")

In [65]:
my_multi_query_retriver = MultiQueryRetriever.from_llm(retriever=retriever, 
                                                      llm=llm,
                                                      prompt=my_multi_query_prompt)

In [66]:
my_multi_query_retriver.invoke(input="毛主席")

[Document(metadata={'description': '毛泽东（1893年12月26日-1976年9月9日），字润之（原作咏芝，后改润芝），笔名子任。湖南湘潭人。中国人民的领袖、伟大的马克思主义者，伟大的无产阶级革命家、战略家、中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人、理论家，是马克思主义中国化的伟大开拓者、中国社会主义现代化建设事业的伟大奠基者，是近代以来中国伟大的爱国者和民族英雄，是党的第一代中央领导集体的核心，是领导中国人民彻底改变自己命运和国家面貌的一代伟人，是为世界被压迫民族的解放和人类进步事业作出重大贡献的伟大国际主义者。1949至1976年，毛泽东担任中华人民共和国最高领导人。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。因毛泽东担任过的主要职务几乎全部称为主席，所以也被人们尊称为“毛主席”。毛泽东被视为现代世界历史中最重要的人物之一，《时代》杂志也将他评为20世纪最具影响1', 'language': 'No language found.', 'source': 'https://baike.baidu.com/item/%E6%AF%9B%E6%B3%BD%E4%B8%9C', 'title': '毛泽东_百度百科'}, page_content='[36]1949至1976年，毛泽东担任中华人民共和国最高领导人。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。 [33-34]因毛泽东担任过的主要职务几乎全部称为主席，所以也被人们尊称为“毛主席”。毛泽东被视为现代世界历史中最重要的人物之一，《时代》杂志也将他评为20世纪最具影响100人之一。中文名毛泽东别\xa0\xa0\xa0\xa0名石三伢子、二十八画生 等国\xa0\xa0\xa0\xa0籍中国民\xa0\xa0\xa0\xa0族汉族出生地湖南省长沙府湘潭县韶山冲出生日期1893年12月26日逝世日期1976年09月09日毕业院校湖南省立第一师范学校代表作品矛盾论、实践论、论持久战、沁园春·雪主要成就领导夺取中国新民主主义革命的胜利在中国确立社会主义基本制度 [22]毛泽东思想的主要创立者信\xa0\xa0\xa0\xa0仰马克思主义目录1人物生平2大事年表▪1893-1910年▪191

### 上下文压缩检索器 LLMChainExtractor
- 上下文可能很大
- 上下文可能包含无用信息
- 增加调用成本
- 潜在好处
    - 提高效率
    - 增强准确性
    - 降低成本

In [67]:
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers.document_compressors import LLMChainFilter
from langchain.retrievers import ContextualCompressionRetriever

In [70]:
# 实例化一个压缩器
compressor = LLMChainExtractor.from_llm(llm=llm)

In [71]:
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor,
                                                       base_retriever=retriever)

In [72]:
results1 = retriever.invoke(input="毛主席是谁？")

In [73]:
results2 = compression_retriever.invoke(input="毛主席是谁？")

In [74]:
len(results1)

10

In [75]:
len(results2)

10

In [76]:
results1[2].page_content

'[1]毛泽东像集（一）大事年表播报编辑1893-1910年1893年12月26日，诞生在湖南省湘潭县韶山冲一个农民家庭。1902年-1909年在家乡韶山六所私塾读书，接受中国传统的启蒙教育。与湘潭学友会合影(二排左三)1910年秋季，考入湖南湘乡县立东山高等小学堂读书。此期间受康有为、梁启超改良主义思想的影响。 [20]1911-1920年1911年春季，到长沙，考入湘乡驻省中学读书。期间，读到同盟会办的《民立报》，受其影响，撰文表示拥护孙中山及同盟会的纲领。10月，响应辛亥革命，投笔从戎，在湖南新军当列兵。半年后退出。'

In [77]:
results2[2].page_content

'[1]毛泽东像集（一）大事年表播报编辑1893-1910年1893年12月26日，诞生在湖南省湘潭县韶山冲一个农民家庭。'

In [78]:
_fileter = LLMChainFilter.from_llm(llm=llm)

In [46]:
compression_filter_retriever = ContextualCompressionRetriever(base_compressor=_fileter,
                                                             base_retriever=retriever)

In [47]:
results1 = retriever.invoke(input="周恩来是谁？")

In [48]:
results2 = compression_filter_retriever.invoke(input="周恩来是谁？")

In [49]:
len(results1)

10

In [50]:
len(results2)

1

In [79]:
for result in results1:
    print(result.page_content, "\n\n\n")

毛泽东_百度百科 网页新闻贴吧知道网盘图片视频地图文库资讯采购百科百度首页登录注册进入词条全站搜索帮助首页秒懂百科特色百科知识专题加入百科百科团队权威合作个人中心收藏查看我的收藏0有用+10毛泽东[máo zé dōng]播报上传视频伟大的无产阶级革命家毛泽东（1893年12月26日-1976年9月9日），字润之（原作咏芝，后改润芝）， [32]笔名子任。湖南湘潭人。中国人民的领袖、伟大的马克思主义者，伟大的无产阶级革命家、战略家、中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人、理论家，是马克思主义中国化的伟大开拓者、中国社会主义现代化建设事业的伟大奠基者，是近代以来中国伟大的爱国者和民族英雄，是党的第一代中央领导集体的核心，是领导中国人民彻底改变自己命运和国家面貌的一代伟人，是为世界被压迫民族的解放和人类进步事业作出重大贡献的伟大国际主义者。 [36]1949至1976年，毛泽东担任中华人民共和国最高领导人。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。 



[36]1949至1976年，毛泽东担任中华人民共和国最高领导人。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。 [33-34]因毛泽东担任过的主要职务几乎全部称为主席，所以也被人们尊称为“毛主席”。毛泽东被视为现代世界历史中最重要的人物之一，《时代》杂志也将他评为20世纪最具影响100人之一。中文名毛泽东别    名石三伢子、二十八画生 等国    籍中国民    族汉族出生地湖南省长沙府湘潭县韶山冲出生日期1893年12月26日逝世日期1976年09月09日毕业院校湖南省立第一师范学校代表作品矛盾论、实践论、论持久战、沁园春·雪主要成就领导夺取中国新民主主义革命的胜利在中国确立社会主义基本制度 [22]毛泽东思想的主要创立者信    仰马克思主义目录1人物生平2大事年表▪1893-1910年▪1911-1920年▪1921-1930年▪1931-1940年▪1941-1950年▪1951-1960年▪1961-1970年▪1971-1976年3主要作品4历史贡献5人物评价人物生平播报编辑毛泽东，1893年12月26日生于湖南韶山 



[1]毛泽东像集（一）大事年表播报编辑1893-1910年1893年12月26日，

In [81]:
results2[0].page_content

'毛泽东（1893年12月26日-1976年9月9日），字润之（原作咏芝，后改润芝）， [32]笔名子任。湖南湘潭人。中国人民的领袖、伟大的马克思主义者，伟大的无产阶级革命家、战略家、中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人、理论家，是马克思主义中国化的伟大开拓者、中国社会主义现代化建设事业的伟大奠基者，是近代以来中国伟大的爱国者和民族英雄，是党的第一代中央领导集体的核心，是领导中国人民彻底改变自己命运和国家面貌的一代伟人，是为世界被压迫民族的解放和人类进步事业作出重大贡献的伟大国际主义者。'

### EmbeddingsFilter

In [82]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

In [83]:
embedding_filter = EmbeddingsFilter(embeddings=embed, similarity_threshold=0.68)

In [84]:
compression_retriever = ContextualCompressionRetriever(base_compressor=embedding_filter, 
                                                       base_retriever=retriever)

In [85]:
# 基础检索器检索出来的内容
results1 = retriever.invoke(input="周恩来是谁？")

In [86]:
# 过滤
results2 = compression_retriever.invoke(input="周恩来是谁？")

In [87]:
len(results1)

10

In [88]:
len(results2)

3

In [89]:
results2[0].metadata

{'description': '毛泽东（1893年12月26日-1976年9月9日），字润之（原作咏芝，后改润芝），笔名子任。湖南湘潭人。中国人民的领袖、伟大的马克思主义者，伟大的无产阶级革命家、战略家、中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人、理论家，是马克思主义中国化的伟大开拓者、中国社会主义现代化建设事业的伟大奠基者，是近代以来中国伟大的爱国者和民族英雄，是党的第一代中央领导集体的核心，是领导中国人民彻底改变自己命运和国家面貌的一代伟人，是为世界被压迫民族的解放和人类进步事业作出重大贡献的伟大国际主义者。1949至1976年，毛泽东担任中华人民共和国最高领导人。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。因毛泽东担任过的主要职务几乎全部称为主席，所以也被人们尊称为“毛主席”。毛泽东被视为现代世界历史中最重要的人物之一，《时代》杂志也将他评为20世纪最具影响1',
 'language': 'No language found.',
 'source': 'https://baike.baidu.com/item/%E6%AF%9B%E6%B3%BD%E4%B8%9C',
 'title': '毛泽东_百度百科'}

In [90]:
results2[0].page_content

'毛泽东_百度百科 网页新闻贴吧知道网盘图片视频地图文库资讯采购百科百度首页登录注册进入词条全站搜索帮助首页秒懂百科特色百科知识专题加入百科百科团队权威合作个人中心收藏查看我的收藏0有用+10毛泽东[máo zé dōng]播报上传视频伟大的无产阶级革命家毛泽东（1893年12月26日-1976年9月9日），字润之（原作咏芝，后改润芝）， [32]笔名子任。湖南湘潭人。中国人民的领袖、伟大的马克思主义者，伟大的无产阶级革命家、战略家、中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人、理论家，是马克思主义中国化的伟大开拓者、中国社会主义现代化建设事业的伟大奠基者，是近代以来中国伟大的爱国者和民族英雄，是党的第一代中央领导集体的核心，是领导中国人民彻底改变自己命运和国家面貌的一代伟人，是为世界被压迫民族的解放和人类进步事业作出重大贡献的伟大国际主义者。 [36]1949至1976年，毛泽东担任中华人民共和国最高领导人。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。'

In [91]:
results2[0].state["embedded_doc"]

[-0.03192613646388054,
 0.0286710225045681,
 -0.015699200332164764,
 -0.004829515237361193,
 0.007615238428115845,
 0.02929888851940632,
 0.018624475225806236,
 -0.01995794288814068,
 0.01825729012489319,
 0.04633037745952606,
 -0.0011460063979029655,
 0.03330191969871521,
 -0.020101899281144142,
 -0.009647886268794537,
 -0.009898141957819462,
 0.0021301170345395803,
 -0.0021207528188824654,
 0.008686570450663567,
 -0.00020159473933745176,
 0.010454761795699596,
 0.002246350981295109,
 0.02164619229733944,
 -0.01160762645304203,
 0.022910693660378456,
 -0.045997001230716705,
 -0.012871863320469856,
 0.015561608597636223,
 0.0024460800923407078,
 0.020756594836711884,
 -0.044625259935855865,
 0.004345875699073076,
 0.024206386879086494,
 0.023410549387335777,
 0.007090319413691759,
 -0.016539620235562325,
 -0.00740452017635107,
 -0.007131773512810469,
 -0.005905163008719683,
 0.008916383609175682,
 0.01495787501335144,
 -0.01101362332701683,
 -0.020685430616140366,
 0.018817156553268433

In [92]:
results2[0].state["query_similarity_score"]

0.6996408124618799

In [93]:
for result in results2:
    print(result.state["query_similarity_score"])

0.6996408124618799
0.6834997929813013
0.6818075467874137


### DocumentCompressorPipeline

In [94]:
# 冗余过滤
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_text_splitters.character import RecursiveCharacterTextSplitter

In [95]:
# 切分器
spliter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)

In [96]:
# 冗余过滤器
# Filter that drops redundant documents by comparing their embeddings.
redundant_filter = EmbeddingsRedundantFilter(embeddings=embed, similarity_threshold=0.8)

In [97]:
# 相关性过滤
relevant_filter = EmbeddingsFilter(embeddings=embed, similarity_threshold=0.66)

In [98]:
pipeline_compressor = DocumentCompressorPipeline(transformers=[spliter, redundant_filter, relevant_filter])

### 完整执行流程：
- 执行基础检索器：检索到10个候选文档
- 执行分割器：分割成更小的58个文档
- 执行冗余过滤器：通过阈值（0.8）判断，剩下8个文档
- 执行相关性过滤器：通过跟问题的阈值判断（0.66），剩余4个文档

In [99]:
compression_retriever = ContextualCompressionRetriever(base_compressor=pipeline_compressor, 
                                                       base_retriever=retriever)

In [100]:
# 基础检索器
results1 = retriever.invoke(input="周恩来是谁？")
len(results1)

10

In [101]:
temp = spliter.split_documents(documents=results1)

In [102]:
len(temp)

58

In [103]:
# 冗余检测
temp1 = redundant_filter.transform_documents(temp)

In [104]:
len(temp1)

8

In [105]:
temp2 = relevant_filter.compress_documents(temp1, query="周恩来是谁？")

In [106]:
len(temp2)

4

In [ ]:
# 流水线检索器
results2 = compression_retriever.invoke(input="周恩来是谁？")
len(results2)

In [ ]:
"""
    展示原始10个文档
"""
for idx, doc in enumerate(results1):
    print(idx, "\n", doc.page_content, "\n\n")

In [192]:
"""
    展示最后4个文档
"""
for idx, doc in enumerate(results2):
    print(idx, "\n", doc.page_content, "\n\n")

0 
 发到天安门广场，献花圈、诗词，悼念周恩来，声讨“四人帮”。毛泽东错误地批准了否定“天安门事件”的报告 


1 
 沫若毛泽东在七届二中全会讲话毛泽东在陕北窑洞毛泽东在上海打乒乓球毛泽东在韶山学校求学时的毛泽东和蒋介石合影(右)1934年10月，参加红一方面军长征。长征途中，1935年1月中共中央政治局在贵州召开扩 


2 
 料私有制的社会主义改造。1954年9月，第一届全国人民代表大会 


3 
 集团的斗争，不让江青、张春桥等夺取最高领导权的野心得逞。1976年9月9日，毛泽东在北京逝世，享年83岁。 


